In [1]:
import sys
sys.path.append("../")

from pathlib import Path
import matplotlib.pyplot as plt
from pyosim import Analogs3dOsim
import ipywidgets as widgets

In [2]:
# Figures
import matplotlib.pyplot as plt
import seaborn as sns

FIG_STYLE = Path('./fig_style.py')
if not FIG_STYLE.is_file():
    !wget --no-check-certificate --content-disposition https://github.com/romainmartinez/cookbook/raw/master/python/fig_style.py
from fig_style import get_style, show
plt.rcParams.update(get_style(style='mrtnz', figsize='two_col'))

In [3]:
DATA_PATH = Path("/home/romain/Downloads/results/mars/1_inverse_kinematic")

trials = [str(ifile) for ifile in DATA_PATH.glob("*.mot")]
data = [Analogs3dOsim().from_mot(itrial) for itrial in trials]
dim = [i for i in range(data[0].shape[1])]

In [11]:
d = data[0]
d.time_normalization().shape

(1, 25, 101)

In [8]:
@widgets.interact(
    d=widgets.SelectMultiple(options=dim, value=dim, description="Dimensions")
)
def update(d=dim):
    for i in range(len(data)):
        plt.plot(data[i][0, d, :].T)
    show()

interactive(children=(SelectMultiple(description='Dimensions', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…